In [38]:
import pickle
import re
from slap_train_pipeline_v2 import SerializableResults


In [34]:
run_dir = "/n/fs/jborz/projects/slap/outputs/2025-12-14/17-10-32"

In [35]:
with open(run_dir + "/results.pkl", "rb") as f:
    results: SerializableResults = pickle.load(f)

In [36]:
results.teleporter_locations

[((1, 0), (2, 1))]

In [ ]:
def analyze_teleporters(results):
    if not results.teleporter_locations:
        print("No teleporter locations found.")
        return

    # 1. Create mapping from (col, row) to node_id
    # Using final_training_data to get node mappings
    data = results.final_training_data
    if not data:
        data = results.training_data
    
    node_atoms = data.node_atoms
    coord_to_node = {}
    
    for node_id, atoms in node_atoms.items():
        col = -1
        row = -1
        for atom in atoms:
            atom_str = str(atom)
            # Match InColX and InRowY
            col_match = re.search(r"InCol(\d+)", atom_str)
            if col_match:
                col = int(col_match.group(1))
            
            row_match = re.search(r"InRow(\d+)", atom_str)
            if row_match:
                row = int(row_match.group(1))
        
        if col != -1 and row != -1:
            coord_to_node[(col, row)] = node_id

    # 2. Get shortcuts sets
    unique_shortcuts_set = set()
    if results.training_data:
        unique_shortcuts_set = set(results.training_data.unique_shortcuts)
    elif results.pruned_training_data: # Fallback
         unique_shortcuts_set = set(results.pruned_training_data.unique_shortcuts)

    final_shortcuts_set = set()
    if results.final_training_data:
        final_shortcuts_set = set(results.final_training_data.unique_shortcuts)

    # 3. Print table
    print(f"\nAnalyzing {len(results.teleporter_locations)} teleporters...")
    print(f"{'Teleporter (Coords)':<30} {'Node IDs':<15} {'In Train?':<10} {'In Prune?':<10} {'In Final?'}")
    print("-" * 80)

    for loc1, loc2 in results.teleporter_locations:
        loc1 = tuple(loc1)
        loc2 = tuple(loc2)
        
        id1 = coord_to_node.get(loc1)
        id2 = coord_to_node.get(loc2)
        
        if id1 is None or id2 is None:
            print(f"{str(loc1):<14}<->{str(loc2):<14} {'Not Found':<15} {'-':<10} {'-':<10} {'-'}")
            continue
            
        in_train = ((id1, id2) in unique_shortcuts_set) or ((id2, id1) in unique_shortcuts_set)
        
        in_prune = False
        if results.pruned_training_data:
            pruned_set = set(results.pruned_training_data.unique_shortcuts)
            in_prune = ((id1, id2) in pruned_set) or ((id2, id1) in pruned_set)
        
        in_final = ((id1, id2) in final_shortcuts_set) or ((id2, id1) in final_shortcuts_set)
        
        coords_str = f"{loc1} <-> {loc2}"
        nodes_str = f"{id1} <-> {id2}"
        
        print(f"{coords_str:<30} {nodes_str:<15} {str(in_train):<10} {str(in_prune):<10} {str(in_final)}")

analyze_teleporters(results)


Analyzing 1 teleporters...
Teleporter (Coords)            Node IDs        In Train?  In Prune?  In Final?
--------------------------------------------------------------------------------
(1, 0) <-> (2, 1)              1 <-> 2         True       True       False
